## Create sklearn and xgboost model

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier

from sklearn import svm
import warnings
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv("mushrooms.csv")
data.head(2)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g


In [6]:
data.dtypes

class                       object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises                     object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

In [7]:
labels = ['Edible', 'Poisonous']
values = [data.describe()['class']['freq'], data.describe()['class']['count']-data.describe()['class']['freq']]
colors = ['green', 'red']

In [15]:
# Encoding the string-type data points

data2 = pd.get_dummies(data)

y = data2[['class_e', 'class_p']]   # The label for the machine learning models
X = data2.drop(['class_e', 'class_p'], axis=1)   #Features

X = X.astype('float32')  # for triton support
trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=13)
trainX.shape, trainY.shape, testX.shape, testY.shape

((6499, 117), (6499, 2), (1625, 117), (1625, 2))

In [29]:
rf = RandomForestClassifier()
rf.fit(trainX, trainY['class_e'])

train_scoreRF = rf.score(trainX, trainY['class_e'])
# oob_score = rf.oob_score_
# Adjusted_R2_trainRF = 1 - (1 - rf.score(trainX, trainY)) * (len(trainY) - 1) / (len(trainY) - trainX.shape[1] - 1)
# Adjusted_R2_testRF = 1 - (1 - rf.score(testX, testY)) * (len(testY) - 1) / (len(testY) - testX.shape[1] - 1)

print('Train Adjusted R2: %', Adjusted_R2_trainRF * 100)
print('Test Adjusted R2: %', Adjusted_R2_testRF * 100)
print('OOB Score: %', oob_score * 100)

Train Adjusted R2: % 100.0
Test Adjusted R2: % 100.0
OOB Score: % 100.0


In [30]:
import pickle
with open('rf.pkl', 'wb') as model_file:
    pickle.dump(rf, model_file)

In [17]:
model = XGBClassifier(learning_rate=0.005, max_depth=10, n_estimators=30,
                      colsample_bytree=0.3, min_child_weight=0.5, reg_alpha=0.3,
                      )
model.fit(trainX, trainY['class_e'])

predictions_XGBC = model.predict(testX)
acc_XGBC = accuracy_score(predictions_XGBC, testY['class_e'])
print('----------------------------------------------------')
print('Accuracy of XGBClassifier: %', 100 * acc_XGBC)

----------------------------------------------------
Accuracy of XGBClassifier: % 100.0


In [21]:
model.save_model('xgboost.model')